In [ ]:
library(tidyverse)
library(DESeq2)

In [ ]:
# TODO: Consider filtering counts (e.g. To remove rows with fewer than 10 reads)
cts <- read_delim(snakemake@input[["counts"]],delim = "\t", skip = 1) %>% 
    rename_with(., ~ stringr::str_extract(pattern = "(?<=\\/)\\w{4}-\\d{1}-\\w+-Seq(?=\\/)", string = .x), contains("out.bam")) %>% 
    column_to_rownames("Geneid") %>% 
    mutate(total_reads = rowSums(select(., contains("Seq")))) %>% 
    filter(total_reads >= 10) %>% 
    dplyr::select(-Chr, -Start, -End, -Strand, -Length, -total_reads, -"AcLi-2-Root-Seq") %>% 
    as.matrix(.)

In [ ]:
head(cts)

In [ ]:
samples <- read_delim(snakemake@config[["sample_sheet"]], delim = "\t") %>% 
    filter(sample != "AcLi-2-Root-Seq")
samples

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = samples,
                              design = ~ cyanotype)
dds

In [ ]:
dds$condition <- relevel(dds$cyanotype, ref = "acli")
dds <- DESeq(dds)

In [ ]:
res <- results(dds)
res

In [ ]:
# TODO: Try different shrinkage estimators
# Note 'apeglm' is the default
# resLFC <- lfcShrink(dds, coef=resultsNames(dds)[2], type="apeglm")
# resLFC

In [ ]:
summary(res)

In [ ]:
plotMA(res)

In [ ]:
plotCounts(dds, gene=which.min(res$padj), intgroup="cyanotype")

In [ ]:
res[which.min(res$padj),]